<a href="https://colab.research.google.com/github/Joracosu/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/AG1/Jos%C3%A9_Ram%C3%B3n_Conde_AG1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Actividad Guiada 1 de Algoritmos de Optimización

José Ramón Conde Suárez

https://colab.research.google.com/drive/1b0od-aP07ucKhmEqv2oQXfaofBG7L6km

### Ejercicio de las torres de Hanói

In [0]:
def torres (N, desde, hasta):
  if N==1:
    print('Lleva la ficha desde', str(desde),'hasta',str(hasta))
  else:
    torres(N-1, desde, 6-desde-hasta)
    print('Lleva la ficha desde', str(desde), 'hasta', str(hasta))
    torres(N-1, 6-desde-hasta, hasta)

# torres(3, 1, 3)

### Ejercicio de las n reinas

In [0]:
def reinas(N, solucion, etapa, j=0):
  for i in range(1, N + 1):
    solucion[etapa] = i
    if es_prometedora(solucion, etapa):
      if etapa == N - 1:
        j = j + 1
        print('Una solución es [',j,']:', solucion)
      else:
        j = reinas(N, solucion, etapa + 1, j)
    solucion[etapa] = 0
  return j

def es_prometedora(SOLUCION, etapa):
  for i in range(etapa + 1):
    if SOLUCION.count(SOLUCION[i]) > 1: return False
    for j in range(i + 1, etapa + 1):
      if abs(i-j) == abs(SOLUCION[i] - SOLUCION[j]): return False
  return True

# print('Total',reinas(8,([0,0,0,0,0,0,0,0]),0),'soluciones')


### Ejercicio de vecinos más cercanos

In [125]:
import random

def puntos_mas_cercanos_bruto(lista):
  minimo=max(lista)-min(lista)
  for i in range(len(lista)):
    for j in range(i+1,len(lista)):
      minimo = min(minimo,abs(lista[i]-lista[j]))
  return minimo

lista_de_numeros = [random.randrange(1,10000000) for x in range(1000)]
print(lista_de_numeros)
print(puntos_mas_cercanos_bruto(lista_de_numeros))


[1211643, 2878370, 435695, 4443906, 544930, 1070216, 9860779, 3085018, 6026731, 7515471, 8569532, 7905904, 1613294, 8437152, 7767873, 4198861, 211889, 9881128, 5268758, 4893950, 3873731, 735519, 5085654, 8191894, 5057529, 1948156, 6251974, 8966635, 3859005, 8037711, 789565, 9714034, 1249534, 6089827, 6141810, 960932, 2578811, 3956604, 1242444, 1695958, 701067, 2884836, 1539374, 9964113, 89411, 1877551, 9943150, 5819351, 4891585, 6443903, 1514263, 3992730, 1237415, 2196340, 7286561, 386573, 1848343, 1517372, 8666087, 690497, 4666201, 7793776, 6011548, 2962682, 144590, 1169880, 32657, 6431925, 3108543, 8887156, 9305828, 5494131, 3583903, 3871017, 9554097, 9091303, 9331869, 893689, 1298657, 4492652, 7267424, 2235566, 6325624, 614217, 9541329, 2071084, 3282805, 9437362, 5980700, 9064096, 6325279, 6367824, 599027, 4816066, 7474069, 497164, 3702305, 9531458, 1235652, 3740161, 9644303, 1864207, 9788846, 9238225, 9852512, 3944005, 817714, 8126881, 8366908, 560632, 548462, 5625290, 4578166, 974